In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)          
        y = np.load(y_path)           
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(np.argmax(y, axis=1), dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class CNNEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1)  
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)  
        x = self.net(x)
        return x.squeeze(-1)   

class FullModel(nn.Module):
    def __init__(self, encoder, num_classes=62):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        emb = self.encoder(x)
        logits = self.classifier(emb)
        return logits

class EarlyStopping:
    def __init__(self, patience=7):
        self.patience = patience
        self.best_acc = 0.0
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.counter = 0
            self.best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

def train_and_eval(model, train_loader, test_loader,
                   epochs=500, patience=7):

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.0005)
    early_stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                preds = model(x).argmax(1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        val_acc = correct / total
        print(f"Epoch {epoch+1:02d} | Val Acc: {val_acc:.4f}")

        if early_stopper.step(val_acc, model):
            print("Early stopping")
            break

    early_stopper.restore(model)
    return early_stopper.best_acc

BASE = "models/Data/Data/62_classes/UserIndependent"

accs = []

for split in range(1, 20):
    print(f"\n===== Split {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128)

    model = FullModel(CNNEncoder(), num_classes=62).to(device)

    acc = train_and_eval(model, train_loader, test_loader)
    accs.append(acc)

    print(f"Best accuracy (split {split}): {acc:.4f} ")

print("\nFINAL AVERAGE ACCURACY (CNN + CCE):", np.mean(accs))


Using device: cuda

===== Split 1 =====
Epoch 01 | Val Acc: 0.1032
Epoch 02 | Val Acc: 0.3613
Epoch 03 | Val Acc: 0.4323
Epoch 04 | Val Acc: 0.4919
Epoch 05 | Val Acc: 0.5097
Epoch 06 | Val Acc: 0.5000
Epoch 07 | Val Acc: 0.5274
Epoch 08 | Val Acc: 0.5823
Epoch 09 | Val Acc: 0.5887
Epoch 10 | Val Acc: 0.6419
Epoch 11 | Val Acc: 0.6355
Epoch 12 | Val Acc: 0.6419
Epoch 13 | Val Acc: 0.5984
Epoch 14 | Val Acc: 0.5823
Epoch 15 | Val Acc: 0.5968
Epoch 16 | Val Acc: 0.6113
Epoch 17 | Val Acc: 0.6274
Early stopping
Best accuracy (split 1): 0.6419 

===== Split 2 =====
Epoch 01 | Val Acc: 0.1387
Epoch 02 | Val Acc: 0.2323
Epoch 03 | Val Acc: 0.3452
Epoch 04 | Val Acc: 0.3452
Epoch 05 | Val Acc: 0.3935
Epoch 06 | Val Acc: 0.3726
Epoch 07 | Val Acc: 0.4403
Epoch 08 | Val Acc: 0.4065
Epoch 09 | Val Acc: 0.4371
Epoch 10 | Val Acc: 0.4484
Epoch 11 | Val Acc: 0.4419
Epoch 12 | Val Acc: 0.4097
Epoch 13 | Val Acc: 0.4742
Epoch 14 | Val Acc: 0.4548
Epoch 15 | Val Acc: 0.4790
Epoch 16 | Val Acc: 0.4774


In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)
        y = np.load(y_path)

        # channel-wise normalization (same as Keras)
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(np.argmax(y, axis=1), dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------- CNN + BiLSTM Model ----------------
class CNN_BiLSTM(nn.Module):
    def __init__(self, num_classes=62):
        super().__init__()

        # CNN part (same filters as Keras)
        self.cnn = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU()
        )

        # BiLSTM (same idea as Keras)
        self.bilstm = nn.LSTM(
            input_size=256,
            hidden_size=256,
            batch_first=True,
            bidirectional=True
        )

        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(512, num_classes)  # 256*2

    def forward(self, x):
        # x: (B, T, C)
        x = x.permute(0, 2, 1)       # (B, C, T)
        x = self.cnn(x)              # (B, 256, T')

        x = x.permute(0, 2, 1)       # (B, T', 256)
        out, _ = self.bilstm(x)      # (B, T', 512)

        feat = out[:, -1, :]         # last timestep
        feat = self.dropout(feat)
        logits = self.classifier(feat)

        return logits

# ---------------- Early Stopping ----------------
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_acc = 0.0
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.counter = 0
            self.best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ---------------- Train & Evaluate ----------------
def train_and_eval(model, train_loader, test_loader,
                   epochs=500, patience=10):

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # evaluation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                preds = model(x).argmax(dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:03d} | Val Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping triggered")
            break

    stopper.restore(model)
    return stopper.best_acc

# ---------------- Main Loop ----------------
BASE = "models/Data/Data/62_classes/UserIndependent"
accs = []

for split in range(1, 20):
    print(f"\n===== SPLIT {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128)

    model = CNN_BiLSTM(num_classes=62).to(device)

    acc = train_and_eval(model, train_loader, test_loader)
    accs.append(acc)

    print(f"Best accuracy (split {split}): {acc:.4f}")

    del model
    torch.cuda.empty_cache()

print("\nFINAL AVERAGE ACCURACY (CNN + BiLSTM + CCE):", np.mean(accs))


Using device: cuda

===== SPLIT 1 =====
Epoch 001 | Val Acc: 0.0355
Epoch 002 | Val Acc: 0.0500
Epoch 003 | Val Acc: 0.1032
Epoch 004 | Val Acc: 0.1839
Epoch 005 | Val Acc: 0.1500
Epoch 006 | Val Acc: 0.2371
Epoch 007 | Val Acc: 0.2500
Epoch 008 | Val Acc: 0.2855
Epoch 009 | Val Acc: 0.4065
Epoch 010 | Val Acc: 0.4484
Epoch 011 | Val Acc: 0.4968
Epoch 012 | Val Acc: 0.4419
Epoch 013 | Val Acc: 0.5210
Epoch 014 | Val Acc: 0.4855
Epoch 015 | Val Acc: 0.5290
Epoch 016 | Val Acc: 0.5242
Epoch 017 | Val Acc: 0.5242
Epoch 018 | Val Acc: 0.4935
Epoch 019 | Val Acc: 0.5452
Epoch 020 | Val Acc: 0.5306
Epoch 021 | Val Acc: 0.5742
Epoch 022 | Val Acc: 0.5823
Epoch 023 | Val Acc: 0.5629
Epoch 024 | Val Acc: 0.5790
Epoch 025 | Val Acc: 0.5887
Epoch 026 | Val Acc: 0.5871
Epoch 027 | Val Acc: 0.5387
Epoch 028 | Val Acc: 0.6016
Epoch 029 | Val Acc: 0.6048
Epoch 030 | Val Acc: 0.5758
Epoch 031 | Val Acc: 0.6081
Epoch 032 | Val Acc: 0.5823
Epoch 033 | Val Acc: 0.6435
Epoch 034 | Val Acc: 0.5871
Epoch 03

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- Dataset ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)                  # (N, T, C)
        y = np.load(y_path)                  # (N, num_classes)

        # per-sample, per-channel normalization
        for i in range(X.shape[0]):
            for c in range(X.shape[2]):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(np.argmax(y, axis=1), dtype=torch.long)
        self.num_classes = y.shape[1]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------- BiLSTM Encoder ----------------
class BiLSTMEncoder(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=64, num_layers=1, dropout=0.3):
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0
        )

    def forward(self, x):
        # x: (B, T, C)
        out, _ = self.lstm(x)          # (B, T, 2*hidden_dim)
        emb = out.mean(dim=1)          # temporal mean pooling ✅
        return emb                     # (B, 2*hidden_dim)

# ---------------- Full Model ----------------
class BiLSTMModel(nn.Module):
    def __init__(self, num_classes, hidden_dim=64):
        super().__init__()
        self.encoder = BiLSTMEncoder(
            input_dim=6,
            hidden_dim=hidden_dim,
            num_layers=1,
            dropout=0.3
        )
        self.classifier = nn.Linear(2 * hidden_dim, num_classes)

    def forward(self, x):
        emb = self.encoder(x)
        logits = self.classifier(emb)
        return logits

# ---------------- Early Stopping ----------------
class EarlyStopping:
    def __init__(self, patience=7):
        self.patience = patience
        self.best_acc = -1
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.counter = 0
            self.best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ---------------- Training ----------------
def train_and_eval(model, train_loader, test_loader,
                   epochs=300, patience=7):

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # -------- Evaluation --------
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                preds = model(x).argmax(dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:03d} | Val Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping")
            break

    stopper.restore(model)
    return stopper.best_acc

# ---------------- Main Loop ----------------
BASE = "models/Data/Data/62_classes/UserIndependent"
accs = []

for split in range(1, 20):
    print(f"\n===== SPLIT {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128)

    model = BiLSTMModel(
        num_classes=train_ds.num_classes,
        hidden_dim=64
    ).to(device)

    acc = train_and_eval(
        model,
        train_loader,
        test_loader
    )

    accs.append(acc)
    print(f"Best accuracy (split {split}): {acc:.4f}")

    del model
    torch.cuda.empty_cache()

print("\nFINAL AVERAGE ACCURACY (BiLSTM):", np.mean(accs))


Using device: cuda

===== SPLIT 1 =====
Epoch 001 | Val Acc: 0.0274
Epoch 002 | Val Acc: 0.0403
Epoch 003 | Val Acc: 0.0484
Epoch 004 | Val Acc: 0.1097
Epoch 005 | Val Acc: 0.1242
Epoch 006 | Val Acc: 0.1097
Epoch 007 | Val Acc: 0.1339
Epoch 008 | Val Acc: 0.1629
Epoch 009 | Val Acc: 0.1661
Epoch 010 | Val Acc: 0.1661
Epoch 011 | Val Acc: 0.1629
Epoch 012 | Val Acc: 0.1661
Epoch 013 | Val Acc: 0.2048
Epoch 014 | Val Acc: 0.2048
Epoch 015 | Val Acc: 0.2419
Epoch 016 | Val Acc: 0.2565
Epoch 017 | Val Acc: 0.2855
Epoch 018 | Val Acc: 0.2984
Epoch 019 | Val Acc: 0.2823
Epoch 020 | Val Acc: 0.2919
Epoch 021 | Val Acc: 0.3000
Epoch 022 | Val Acc: 0.3274
Epoch 023 | Val Acc: 0.2984
Epoch 024 | Val Acc: 0.3113
Epoch 025 | Val Acc: 0.3258
Epoch 026 | Val Acc: 0.3194
Epoch 027 | Val Acc: 0.3274
Epoch 028 | Val Acc: 0.3613
Epoch 029 | Val Acc: 0.3677
Epoch 030 | Val Acc: 0.3710
Epoch 031 | Val Acc: 0.3516
Epoch 032 | Val Acc: 0.3871
Epoch 033 | Val Acc: 0.3710
Epoch 034 | Val Acc: 0.3871
Epoch 03

In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ===================== Dataset =====================
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path, downsample=4):
        X = np.load(x_path)          # (N, T, C)
        y = np.load(y_path)          # (N, num_classes)

        # ✅ downsample time axis (CRITICAL for BiLSTM)
        X = X[:, ::downsample, :]

        # ❌ NO per-sample normalization (very important)
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(np.argmax(y, axis=1), dtype=torch.long)
        self.num_classes = y.shape[1]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ===================== BiLSTM Encoder =====================
class BiLSTMEncoder(nn.Module):
    def __init__(self, input_dim=6, proj_dim=32, hidden_dim=64):
        super().__init__()

        # ✅ input projection (stabilizes learning)
        self.input_proj = nn.Linear(input_dim, proj_dim)

        self.lstm = nn.LSTM(
            input_size=proj_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )

    def forward(self, x):
        # x: (B, T, 6)
        x = self.input_proj(x)           # (B, T, 32)
        out, _ = self.lstm(x)            # (B, T, 128)
        emb = out.mean(dim=1)            # temporal mean pooling ✅
        return emb                       # (B, 128)

# ===================== Full Model =====================
class BiLSTMModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.encoder = BiLSTMEncoder()
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        emb = self.encoder(x)
        logits = self.classifier(emb)
        return logits

# ===================== Early Stopping =====================
class EarlyStopping:
    def __init__(self, patience=7):
        self.patience = patience
        self.best_acc = -1
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.counter = 0
            self.best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ===================== Training =====================
def train_and_eval(model, train_loader, test_loader,
                   epochs=300, patience=7):

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # ---- Evaluation ----
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                preds = model(x).argmax(dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:03d} | Val Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping")
            break

    stopper.restore(model)
    return stopper.best_acc

# ===================== Main Loop =====================
BASE = "models/Data/Data/62_classes/UserIndependent"
accs = []

for split in range(1, 20):
    print(f"\n===== SPLIT {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    # ✅ smaller batch size for LSTM
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=64)

    model = BiLSTMModel(num_classes=train_ds.num_classes).to(device)

    acc = train_and_eval(model, train_loader, test_loader)
    accs.append(acc)

    print(f"Best accuracy (split {split}): {acc:.4f}")

    del model
    torch.cuda.empty_cache()

print("\nFINAL AVERAGE ACCURACY (PATCHED BiLSTM):", np.mean(accs))


Using device: cuda

===== SPLIT 1 =====
Epoch 001 | Val Acc: 0.0419
Epoch 002 | Val Acc: 0.0387
Epoch 003 | Val Acc: 0.0597
Epoch 004 | Val Acc: 0.0823
Epoch 005 | Val Acc: 0.1032
Epoch 006 | Val Acc: 0.1000
Epoch 007 | Val Acc: 0.1097
Epoch 008 | Val Acc: 0.1097
Epoch 009 | Val Acc: 0.1081
Epoch 010 | Val Acc: 0.1226
Epoch 011 | Val Acc: 0.1468
Epoch 012 | Val Acc: 0.1387
Epoch 013 | Val Acc: 0.1661
Epoch 014 | Val Acc: 0.1645
Epoch 015 | Val Acc: 0.1677
Epoch 016 | Val Acc: 0.1790
Epoch 017 | Val Acc: 0.1742
Epoch 018 | Val Acc: 0.1710
Epoch 019 | Val Acc: 0.1839
Epoch 020 | Val Acc: 0.2016
Epoch 021 | Val Acc: 0.1984
Epoch 022 | Val Acc: 0.2129
Epoch 023 | Val Acc: 0.2258
Epoch 024 | Val Acc: 0.2145
Epoch 025 | Val Acc: 0.2177
Epoch 026 | Val Acc: 0.2290
Epoch 027 | Val Acc: 0.2371
Epoch 028 | Val Acc: 0.2387
Epoch 029 | Val Acc: 0.2355
Epoch 030 | Val Acc: 0.2581
Epoch 031 | Val Acc: 0.2532
Epoch 032 | Val Acc: 0.2661
Epoch 033 | Val Acc: 0.2484
Epoch 034 | Val Acc: 0.2823
Epoch 03

KeyboardInterrupt: 